[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/openai/chatgpt/plugins/ask-lex/ask-lex-indexer.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/openai/chatgpt/plugins/ask-lex/ask-lex-indexer.ipynb)

# Indexing for the Ask Lex ChatGPT Plugin

This notebook works through the indexing process for processing videos from a YouTube channel (in this case Lex Fridman) and storing them inside a Pinecone vector DB to be used by a retrieval ChatGPT plugin.

To begin we install prerequisite libraries and setup our API keys.

In [ ]:
!pip install -qU openai pod-gpt datasets git+https://github.com/openai/whisper.git

Now enter API keys. Note that this will cost some money for creating the embeddings via OpenAI unless within their free usage credits.

In [ ]:
import os

# get openai api key at platform.openai.com
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "OPENAI_API_KEY"
# get pinecone api key at app.pinecone.com
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"
# find your environment next to the api key in pinecone console
PINECONE_ENV = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"

## Downloading and Transcribing (Optional)

This section is completely optional as you can just jump ahead to the next section and use the prebuilt `jamescalam/lex-transcripts` dataset. If choosing to run this section note that it will likely take multiple days.

In [ ]:
import pod_gpt

channel = pod_gpt.Channel(
    channel_id='UCSHZKyawb77ixDdsGog4iWA',  # lex fridman YT channel
    api_key=YOUTUBE_API_KEY  # Google YouTube API
)

You can return a specific number of videos by setting `max_results` parameter. To return **all** videos just remove the parameter to use `channel.get_videos_info()`.

In [ ]:
channel.get_videos_info(max_results=2)

Load the Whisper audio transcription model:

In [ ]:
import torch
import whisper

# prep whisper model
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = whisper.load_model("large").to(device)

Transcribe audio:

In [ ]:
channel.transcribe_videos(model)

Save transcribed audio to local JSONL file:

In [ ]:
channel.save()

## Using Prebuilt Dataset

Rather than going through the process above, we can skip ahead a little with a prebuilt Lex dataset from Hugging Face. We load it like so:

In [ ]:
from datasets import load_dataset

data = load_dataset(
    'jamescalam/lex-transcripts',
    split='train'
)
data

Dataset({
    features: ['video_id', 'channel_id', 'title', 'published', 'transcript', 'source'],
    num_rows: 499
})

## Indexing

Now that we have the dataset ready we can begin indexing it in Pinecone using OpenAI's `text-embedding-ada-002` model. To begin we initialize a `pod_gpt` `indexer`:

In [ ]:
import pod_gpt

indexer = pod_gpt.Indexer(
    openai_api_key=OPENAI_API_KEY,
    pinecone_api_key=PINECONE_API_KEY,
    pinecone_environment=PINECONE_ENV,
    index_name="ask-lex"
)

Now we index our data:

In [ ]:
from tqdm.auto import tqdm

for row in tqdm(data):
    row['published'] = row['published'].strftime('%Y%m%d')
    indexer(pod_gpt.VideoRecord(**row))

  0%|          | 0/499 [00:00<?, ?it/s]

Once complete we can move on to building the remainder of the plugin. Please see [this video](https://youtu.be/bAQ6VRewf0w) for a more detailed walkthrough.